<a href="https://colab.research.google.com/github/chenyq121/760-2022S2/blob/main/KFoldOptunaBaselineCNNTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 35.8 MB/s 
     |████████████████████████████████| 209 kB 70.7 MB/s 
     |████████████████████████████████| 81 kB 10.1 MB/s 
     |████████████████████████████████| 78 kB 9.2 MB/s 
     |████████████████████████████████| 147 kB 71.4 MB/s 
     |████████████████████████████████| 112 kB 73.5 MB/s 
     |████████████████████████████████| 49 kB 5.2 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=fcc2aec39cf5f5cb9b88157f618fd978b69c848b2e4c0130b2f61b593076642f
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 5.0.0
    Uninstalling importlib-metadata-5.0.0:
      Successfully uninstalled importlib-metadata-5.0.0


In [ ]:
#mounted to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

os.chdir('/content/drive/Shareddrives/760/Data/')
os.getcwd()
os.listdir('.')

['image_Bored_Ape.zip',
 'Last_Sale_Price.csv',
 'price_cleaned.csv',
 'Bored_Ape_Dataset_Analysis.html',
 'ape',
 '1',
 '2',
 'temphold',
 'ape copy',
 'all_properties.csv',
 'test.txt',
 'validate.txt',
 'train.txt',
 'all_properties.gsheet',
 'price_cleaned_under200.csv',
 'u1.txt',
 'u2.txt',
 'test',
 'train',
 'u1',
 'u2',
 'utrain.txt',
 'uvalidate.txt',
 'utest.txt',
 'Trantimes.csv',
 'Tranaction.csv',
 'TransactionData.html',
 'price_cleaned_under200_log_label.numbers',
 'log-bin.gsheet',
 'log-bin.xlsx',
 'pricewlabel.csv',
 'cs760-model result track.xlsx',
 'ID-price-label.xlsx',
 'grey',
 'to_archive_760project data pre-process.ipynb',
 'Copy of test (1).txt',
 'Copy of train (1).txt',
 'Copy of test.txt',
 'Copy of train.txt',
 'testopt.txt',
 'trainopt.txt']

In [ ]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torchvision
import os
import torch.utils.data
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import KFold

import optuna

# data processing 
class ClassifyDataset(torch.utils.data.Dataset):
    def __init__(self,root_path,data_file,img_size=120):
        self.data_files=np.loadtxt(data_file,dtype=np.str)
        self.root_path=root_path
        self.class_list=os.listdir(
            os.path.join(root_path,'')
        )
        self.transforms=torchvision.transforms.Compose(
            [
                torchvision.transforms.Resize((img_size,img_size)), 
                torchvision.transforms.ToTensor()
            ]
        )

    def __getitem__(self, item):
        data_file=self.data_files[item]
        data_file=os.path.join(self.root_path,data_file)
        # get the image
        img=Image.open(data_file).convert('RGB') # three channels
        # get the label(in this case the label is the folder name)
        tmp=data_file.split('/')
        label_name=tmp[-2]
        #print("label_name:",label_name)
        label=self.class_list.index(label_name)
        #print("label:",label)
        # prepare the images and label
        img=self.transforms(img)
        label=torch.tensor(label)
        #print("torch_label:",label)
        return img,label

    def __len__(self):
        return len(self.data_files)



# model building
class CNet(nn.Module):
    def __init__(self,num_classes=21, n_layer = 1, kernel_size = 1):
        super(CNet,self).__init__()
        self.convList = nn.ModuleList()

        for i in range(1, n_layer):
          input = 2 ** (i + 3)
          output = 2 ** (i + 4)
          if i == 1:
            input = 3
            
          self.convList.append(
              nn.Sequential(
                #input, output, kernel size, step, padding
                nn.Conv2d(input, output, kernel_size, 1, padding=1),
                nn.BatchNorm2d(output), # batch normalization
                nn.ReLU()
              )
          )

        self.pool = nn.AvgPool2d(2, 2)
        self.fclayer=nn.Sequential(
                nn.Linear(2 * input, 2 * output),
                nn.ReLU(),
                nn.Linear(2 * output, num_classes)
        )
        self.avg_pool=nn.AdaptiveAvgPool2d((1, 1))
        self.softmax=nn.Softmax(dim=1)

    def forward(self,x):
      x = self.convList[0](x)
      x = self.pool(x)
      for conv in self.convList[1:]:
        x = conv(x)
        x = self.pool(x)
      x = self.avg_pool(x)
      x = torch.flatten(x,1)
      logits=self.fclayer(x)
      prob=self.softmax(logits)
      return logits,prob

    


# training
def training(model, train_loader, len_train, batch_size,lr, epoch_num):
    # get training data
    #train_dataset=ClassifyDataset(root_path,train_data_file)
    #train_dataloader=torch.utils.data.DataLoader(train_dataset,batch_size,shuffle=True,num_workers=0)


    # loss function and optimizer
    criterion=nn.CrossEntropyLoss()
    optimizer=torch.optim.Adam(model.parameters(),lr=lr)

    total_loss=[]

    for epoch in range(epoch_num):
        # print(epoch+1,"epoch:")
        total_train_loss=0
        res_num=len_train
        for data in train_loader:

            if (res_num - batch_size) > 0:
                cnt=batch_size
                res_num = res_num - batch_size
            else:
                cnt=res_num
                res_num = 0

            ## GPU
            train_img,train_label=data
            train_img=train_img.to(device)
            train_label=train_label.to(device)

            ## get loss result
            train_logits,train_prob=model(train_img)
            train_loss=criterion.forward(train_logits,train_label)

            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()
            
            train_pred=torch.argmax(train_prob,dim=1)
            train_acc=(train_pred==train_label).float()
            train_acc=torch.mean(train_acc)
            # print('loss:',train_loss.item(), 'acc:', train_acc.item())
            total_train_loss=total_train_loss+train_loss*cnt

        total_train_loss=total_train_loss/len_train
        total_loss.append(total_train_loss.item())

    state_dict=model.state_dict()
    torch.save(state_dict,'/content/drive/Shareddrives/760/model/60-20-20model_20_epoch_120_lr0.01-optune_test.pth')

    # plt.xlabel('epoch')
    # plt.ylabel('loss')
    # plt.plot(total_loss)
    # plt.legend(['train loss'])
    # plt.show()

# test
def test(model, valid_loader, len_test):
    state_dict = torch.load('/content/drive/Shareddrives/760/model/60-20-20model_20_epoch_120_lr0.01-optune_test.pth')
    model.load_state_dict(state_dict)
    model.eval() # no BatchNormalization and Dropout

    #test_dataset=ClassifyDataset(root_path,test_data_file)
    #test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size, shuffle=True, num_workers=0)

    criterion = nn.CrossEntropyLoss()

    res_num=len_test
    total_acc=0

    for data in valid_loader:
      with torch.no_grad():

        if (res_num-batch_size)>0:
            cnt=batch_size
            res_num = res_num - batch_size
        else:
            cnt=res_num
            res_num = 0

        test_img,test_label=data
        test_img=test_img.to(device)
        test_label=test_label.to(device)

        test_logits,test_prob=model(test_img)
        test_loss=criterion.forward(test_logits,test_label)

        # Top1 accuracy: correct number/total number
        test_pred = torch.argmax(test_prob, dim=1)
        # print(test_pred)
        # print(test_label)
        test_acc = (test_pred == test_label).float()
        # print(test_acc)
        test_acc = torch.mean(test_acc)
        total_acc = total_acc + test_acc * cnt

        # print('loss:',test_loss.item(), 'top1:',test_acc.item() )

    total_acc=total_acc/len_test
    return total_acc.item()
    # print('\n')
    # print('Top-1 Accuracy:',total_acc.item())


def objective(trial, train_loader, valid_loader, len_train, len_test):

    params = {
              'n_layer': trial.suggest_int('n_layer', 3, 5),
              'kernel_size': trial.suggest_int('kernel_size', 3, 5, step = 2),
              'lr': trial.suggest_categorical('lr', [0.01, 0.05]),#trial.suggest_float('lr', 0.01, 0.1, step = {0.04, 0.05}),
              'epoch_num': trial.suggest_int('epoch_num', 5, 10, step = 5)
              }

    # get model and put model on the device
    model=CNet(n_layer = params['n_layer'], kernel_size = params['kernel_size'])
    model = model.to(device)

    training(model, train_loader, len_train, batch_size,lr = params['lr'], epoch_num = params['epoch_num'])
    # torch.cuda.empty_cache()
    return test(model, valid_loader, len_test)

def objective_cv(trial):

    # get training data
    dataset=ClassifyDataset(root_path,train_data_file)

    fold = KFold(n_splits=3, shuffle=True, random_state=0)
    scores = []
    for fold_idx, (train_idx, valid_idx) in enumerate(fold.split(range(len(dataset)))):
        train_data = torch.utils.data.Subset(dataset, train_idx)
        valid_data = torch.utils.data.Subset(dataset, valid_idx)

        train_loader = torch.utils.data.DataLoader(
            train_data,
            batch_size,
            shuffle=True,
            num_workers=0
        )
        valid_loader = torch.utils.data.DataLoader(
            valid_data,
            batch_size,
            shuffle=True,
            num_workers=0
        )
        accuracy = objective(trial, train_loader, valid_loader, len(train_data), len(valid_data))
        scores.append(accuracy)
    print(scores)
    return np.std(scores)




In [ ]:
# main function
if __name__ == '__main__':

    root_path = r'/content/drive/Shareddrives/760/Data/'
    train_data_file=r'/content/drive/Shareddrives/760/Data/trainopt.txt'
    #test_data_file=r'/content/drive/Shareddrives/760/Data/testopt.txt'
    batch_size=64
    device = torch.device("cuda:0")
    
    study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler())
    study.optimize(objective_cv, n_trials=2)

[I 2022-10-12 04:16:11,725] A new study created in memory with name: no-name-040d2ad5-176e-4328-b9e1-b3172ddc061f
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
[I 2022-10-12 04:16:13,784] Trial 0 finished with value: 0.17817417985996767 and parameters: {'n_layer': 4, 'kernel_size': 3, 'lr': 0.01, 'epoch_num': 5}. Best is trial 0 with value: 0.17817417985996767.


[0.857142984867096, 0.7142857313156128, 0.428571492433548]


[I 2022-10-12 04:16:16,065] Trial 1 finished with value: 0.17817417985996767 and parameters: {'n_layer': 5, 'kernel_size': 3, 'lr': 0.05, 'epoch_num': 5}. Best is trial 0 with value: 0.17817417985996767.


[0.857142984867096, 0.7142857313156128, 0.428571492433548]


In [ ]:
best_trial = study.best_trial

for key, value in best_trial.params.items():
    print("{}: {}".format(key, value))

n_layer: 4
kernel_size: 5
lr: 0.1
epoch_num: 10


In [ ]:
optuna.visualization.plot_optimization_history(study)